In [4]:
import os
import sys
src_path = os.path.abspath('../src')
sys.path.append(src_path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from src import data
from src import portfolio
from src import efficient_frontier
from src import plot
import yfinance as yf
from scipy.optimize import Bounds, LinearConstraint
import statsmodels.api as smf

In [5]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1100
max_esg_score = 2000
df = pd.read_excel(r"../data/ESG_DATA_S&P500.xlsx")

dates = ['2021-01-01','2022-01-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.02

In [7]:
esg_data = data.esg_score_weight(df,weights,min_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
prices = prices[['MMM','AOS','ABT','ACN','ADM','ADP']]
esgdata = esgdata[['MMM_weighted','AOS_weighted','ABT_weighted','ACN_weighted','ADM_weighted','ADP_weighted']]

pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,dates[0],dates[1], Wanted_return, maximum_risk) #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters)


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:40: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,
c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\efficient_frontier.py:113: OptimizeWarning: Unknown solver options: xtol, gtol, barrier_tol
  result = minimize(function,


Min. Risk = 4.075% => Return: 2.453%  Sharpe Ratio = 0.60
Max. Sharpe Ratio = 0.92 => Return: 4.21%  Risk: 4.573%


In [8]:
pct_returns


,MMM,AOS,ABT,ACN,ADM,ADP
Date,,,,,,
2021-02-01 00:00:00-05:00,-0.003416,0.093370,-0.030828,0.037120,0.131374,0.053900
2021-03-01 00:00:00-05:00,0.100651,0.138791,0.000501,0.101036,0.007423,0.083036
2021-04-01 00:00:00-04:00,0.023147,0.002071,0.002003,0.049665,0.107544,-0.007853
2021-05-01 00:00:00-04:00,0.029928,0.049004,-0.028564,-0.026934,0.053857,0.048291
2021-06-01 00:00:00-04:00,-0.021720,0.013930,-0.006172,0.044762,-0.089133,0.013264
2021-07-01 00:00:00-04:00,-0.003474,-0.024008,0.043561,0.077648,-0.014521,0.055433
2021-08-01 00:00:00-04:00,-0.016166,0.033983,0.044553,0.059431,0.004689,-0.002815
2021-09-01 00:00:00-04:00,-0.099209,-0.160204,-0.065205,-0.049441,0.000167,-0.043628
2021-10-01 00:00:00-04:00,0.018584,0.196496,0.091086,0.121499,0.070488,0.122899


In [9]:

sp500 = data.data_for_beta(['SPY','MMM','AOS','ABT','ACN','ADM','ADP'],dates)
 #Gives us the weights of the individual stocks in our portfolio
#portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
#portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
#cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.


In [10]:
sp500

pct_returns_sp500 = data.pct_returns_from_prices(sp500)
pct_returns_sp500.values

array([[ 2.78054904e-02, -3.41572407e-03,  9.33701614e-02,
        -3.08277180e-02,  3.71196916e-02,  1.31373800e-01,
         5.39002508e-02],
       [ 4.19865439e-02,  1.00651179e-01,  1.38790666e-01,
         5.00897973e-04,  1.01036296e-01,  7.42308511e-03,
         8.30364130e-02],
       [ 5.29104583e-02,  2.31472228e-02,  2.07069055e-03,
         2.00271613e-03,  4.96651628e-02,  1.07543878e-01,
        -7.85268593e-03],
       [ 6.56606957e-03,  2.99279391e-02,  4.90036855e-02,
        -2.85642926e-02, -2.69338122e-02,  5.38570831e-02,
         4.82913445e-02],
       [ 1.90933931e-02, -2.17198018e-02,  1.39298982e-02,
        -6.17232071e-03,  4.47618540e-02, -8.91327283e-02,
         1.32639057e-02],
       [ 2.44124942e-02, -3.47380771e-03, -2.40077128e-02,
         4.35607956e-02,  7.76484395e-02, -1.45214072e-02,
         5.54325347e-02],
       [ 2.97598401e-02, -1.61664995e-02,  3.39826436e-02,
         4.45528124e-02,  5.94308906e-02,  4.68852601e-03,
        -2.8145379

In [37]:


def betas_ofportfolio(portfolio, portfolio_weights):
    betas = []
    for i in range(len(portfolio.columns)):

        betas.append(np.cov(pct_returns_sp500[portfolio.columns[i]],pct_returns_sp500['SPY'])[0][1]/np.var(pct_returns_sp500['SPY']))

    k = np.multiply(betas,portfolio_weights).values.sum()

    return(k)        

betas_ofportfolio(prices,weights_of_each_portfolio)


1.3145447035634206

In [39]:
def capm(market,risk_free_rate,beta):
    print(f'Excpected return on investment is {100*(risk_free_rate+beta*(market-risk_free_rate))}%')

    return(risk_free_rate+beta*(market-risk_free_rate))

capm(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_ofportfolio(prices,weights_of_each_portfolio))

def alpha(expected_return,actual_return):
    return(actual_return-expected_return)

 #Gives us the weights of the individual stocks in our portfolio
#portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
#portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
#cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.


Excpected return on investment is 2.449309757437116%


0.02449309757437116